In [4]:
from variables_globales import *
from librerias import *
from funciones import *

### Carga de información

In [5]:
df_luisa = pd.read_excel("a_insumos\\luisa_sent_manual_full_justified.xlsx")\
             [["texto_limpio","sent_manual"]]
df_luisa["canidato"] = "luisa"
df_noboa = pd.read_excel("a_insumos\\noboa_sent_manual_full_justified.xlsx")\
             [["texto_limpio","sent_manual"]]
df_noboa["canidato"] = "noboa"
df = pd.concat([df_luisa,df_noboa])
df.head(2)

,texto_limpio,sent_manual,canidato
0,correistas,NEU,luisa
1,no mientas rana rene a qué vuelves a españa al...,NEG,luisa


In [6]:
print(df.shape)
df = df.dropna(subset=['texto_limpio'])
print(df.shape)
df['texto_limpio'] = df['texto_limpio'].str.lower().str.strip()

(13116, 3)
(13116, 3)


### Entrenamiento Modelos

#### División en test y training

In [ ]:
X_train, X_test, y_train, y_test, vectorizer, encoder = preparar_datos(df)
num_clases = len(encoder.classes_)

#### XGboost

In [10]:
xgb = entrenar_xgboost(X_train, y_train, num_clases)
evaluar_modelo(xgb, X_test, y_test, encoder)

              precision    recall  f1-score   support

         NEG       0.78      0.90      0.84      1629
         NEU       0.58      0.44      0.50       696
         POS       0.71      0.48      0.57       299

    accuracy                           0.73      2624
   macro avg       0.69      0.61      0.64      2624
weighted avg       0.72      0.73      0.72      2624

[[1471  141   17]
 [ 347  307   42]
 [  74   82  143]]


#### Random Forest

In [11]:
rf = entrenar_random_forest(X_train, y_train)
evaluar_modelo(rf, X_test, y_test, encoder)

              precision    recall  f1-score   support

         NEG       0.74      0.93      0.82      1629
         NEU       0.63      0.39      0.48       696
         POS       0.68      0.38      0.49       299

    accuracy                           0.72      2624
   macro avg       0.69      0.57      0.60      2624
weighted avg       0.71      0.72      0.70      2624

[[1507  105   17]
 [ 390  270   36]
 [ 129   55  115]]


#### Red neuronal

In [12]:
rn = entrenar_red_neuronal(X_train, y_train, num_clases)
y_pred_rn = predecir_red_neuronal(rn, X_test, num_clases)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_rn, target_names=encoder.classes_))
print(confusion_matrix(y_test, y_pred_rn))

C:\Users\Cristian Sanchez\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


              precision    recall  f1-score   support

         NEG       0.76      0.95      0.84      1629
         NEU       0.61      0.35      0.45       696
         POS       0.77      0.46      0.57       299

    accuracy                           0.74      2624
   macro avg       0.71      0.59      0.62      2624
weighted avg       0.72      0.74      0.71      2624

[[1547   77    5]
 [ 414  246   36]
 [  82   80  137]]
